In [6]:
import gdal
import numpy as np

#### create your own population count manually

In [7]:
# read in population count
path = '/nfs/a68/earlacoa/population/count/'
population_tif = gdal.Open(
    path + 'gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-2015-qtr-deg.tif'
)
population_count = population_tif.ReadAsArray()

# transform lat and lon
geo_transform = population_tif.GetGeoTransform()
population_x = np.linspace(
    geo_transform[0], 
    geo_transform[0] + geo_transform[1] * population_count.shape[1], 
    population_count.shape[1]
)
population_y = np.linspace(
    geo_transform[3], 
    geo_transform[3] + geo_transform[5] * population_count.shape[0], 
    population_count.shape[0]
)
population_xx, population_yy = np.meshgrid(population_x, population_y)

# replace negative fill values
population_count[population_count < 0] = 0.0

# count is upside down in latitude
population_count = np.flipud(population_count)

# save
np.savez_compressed(
    path + 'gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-2015-qtr-deg.npz', 
    population_count=population_count,
    population_xx=population_xx,
    population_yy=population_yy
)

#### check this manual version is the same as the imported .npz version

In [16]:
def import_npz(npz_file, namespace):
    '''load all numpy arrays into global namespace'''
    '''ensure original arrays have the variable name you require'''
    data = np.load(npz_file)
    for var in data:
        if data[var].dtype == np.dtype('float64'):
            namespace[var] = data[var].astype('float32')
        else:
            namespace[var] = data[var]

In [17]:
# import pop_z_2015
import_npz('/nfs/a68/earlacoa/population/count/population-count-0.25deg.npz', globals())

In [34]:
print((pop_z_2015 == population_array).all())

True


In [36]:
import pandas as pd

In [39]:
#df_baseline_mortality = pd.read_csv('/nfs/a68/earlacoa/health/GBD2017/GBD2017_baseline_mortality_2017_global_all-cause_deaths_rateper100000.csv')
df_bm = pd.read_csv('/nfs/a68/earlacoa/health/GBD2017/GBD2017_BM_2015_countries_cause_deaths-DALYS-YLL-YLD_rateper100000_LRI-LC-COPD-IHD-DIAB-STR-NCD-ALL.csv')

location = 163
baseline_mortality_annual = float(df_bm.loc[df_bm['location_id'] == location][df_bm['year'] == 2015][df_bm['age_id'] == 22][df_bm['cause_id'] == 294]['val']) / 100000
baseline_mortality_daily = baseline_mortality_annual / 365.25

<ipython-input-39-d519490c6f21>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  baseline_mortality_annual = float(df_bm.loc[df_bm['location_id'] == location][df_bm['year'] == 2016][df_bm['age_id'] == 22][df_bm['cause_id'] == 294]['val']) / 100000


TypeError: cannot convert the series to <class 'float'>

In [47]:
df_bm.loc[df_bm['location_id'] == location][df_bm['year'] == 2015][df_bm['age_id'] == 22][df_bm['cause_id'] == 294][df_bm['meausre_id'] == 1]

<ipython-input-47-170d444cd514>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_bm.loc[df_bm['location_id'] == location][df_bm['year'] == 2015][df_bm['age_id'] == 22][df_bm['cause_id'] == 294]


,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,metric_id,metric_name,year,val,upper,lower
42698,1,Deaths,163,India,1,Male,22,All Ages,294,All causes,3,Rate,2015,740.537144,753.410451,727.684187
42699,1,Deaths,163,India,2,Female,22,All Ages,294,All causes,3,Rate,2015,691.967271,705.657281,679.796729
42700,1,Deaths,163,India,3,Both,22,All Ages,294,All causes,3,Rate,2015,716.897717,727.696148,706.784869
191776,4,YLLs (Years of Life Lost),163,India,1,Male,22,All Ages,294,All causes,3,Rate,2015,26033.578280,26540.273910,25533.448400
191777,4,YLLs (Years of Life Lost),163,India,2,Female,22,All Ages,294,All causes,3,Rate,2015,23396.079260,23904.296460,22922.069350
191778,4,YLLs (Years of Life Lost),163,India,3,Both,22,All Ages,294,All causes,3,Rate,2015,24749.881970,25233.927700,24300.118880
375246,3,YLDs (Years Lived with Disability),163,India,1,Male,22,All Ages,294,All causes,3,Rate,2015,10021.692360,12907.845180,7539.290826
375247,3,YLDs (Years Lived with Disability),163,India,2,Female,22,All Ages,294,All causes,3,Rate,2015,12406.828050,15938.934500,9303.467735
375248,3,YLDs (Years Lived with Disability),163,India,3,Both,22,All Ages,294,All causes,3,Rate,2015,11182.560990,14407.599380,8387.335963
559798,2,DALYs (Disability-Adjusted Life Years),163,India,1,Male,22,All Ages,294,All causes,3,Rate,2015,36055.270640,38913.448110,33487.817040


In [ ]:
716.897717	727.696148	706.784869